# Neural Machine Translation (seq2seq): From English to French
This file is modified from the original file named "lstm_seq2seq.py", downloaded from [keras-examples](https://github.com/keras-team/keras/tree/master/examples).

Other reference
- [tensorflow-nmt](https://github.com/tensorflow/nmt).
- [attention-based-sequence-to-sequence-in-keras](https://wanasit.github.io/attention-based-sequence-to-sequence-in-keras.html).

Jun 24, 2019.

guorui233@outlook.com

+ [keras.layers.LSTM](https://github.com/keras-team/keras/blob/master/keras/layers/recurrent.py#L2051) is a class and will define a LSTM layer instead of a cell.

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from time import time
import numpy as np
import io
time_begin = time()

Using TensorFlow backend.


## 1 - Prepare Data

In [2]:
latent_dim = 256  # Dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path = 'fra.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with io.open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
for line in lines[:num_samples]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

('Number of samples:', 10000)
('Number of unique input tokens:', 70)
('Number of unique output tokens:', 93)
('Max sequence length for inputs:', 16)
('Max sequence length for outputs:', 59)


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# len(input_texts)=10000, max_encoder_seq_length=16, num_encoder_tokens=70
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

# len(input_texts)=10000, max_decoder_seq_length=59, num_decoder_tokens=93
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# len(input_texts)=10000, max_decoder_seq_length=59, num_decoder_tokens=93
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

## 2 - Build and Train the Seq2seq Model

In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # num_encoder_tokens=70
encoder = LSTM(units=latent_dim, return_state=True) # latent_dim = 256
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens)) # num_decoder_tokens=93
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# num_decoder_tokens=93, from latent_dim=256 to 93
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [9]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=3,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/3
8000/8000 [==============================] - 9s 1ms/step - loss: 0.9208 - val_loss: 0.9452
Epoch 2/3
8000/8000 [==============================] - 8s 963us/step - loss: 0.7356 - val_loss: 0.7660
Epoch 3/3
8000/8000 [==============================] - 8s 957us/step - loss: 0.6222 - val_loss: 0.6856


In [10]:
# Save model
model.save('s2s.h5')

/home/io/tensorflow/local/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


## 3 - Inference on New Data

In [11]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models

In [12]:
encoder_model = Model(encoder_inputs, encoder_states)

In [13]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
dense_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, 
                      [dense_outputs] + decoder_states)

In [14]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [15]:
def decode_sequence(input_seq): # input_seq is tensor of shape (1, 16, 70)
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character '\t'.
    target_seq[0, 0, target_token_index['\t']] = 1. # '\t' means tab, target_token_index['\t']=0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [16]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
('Input sentence:', u'Go.')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Hi.')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Run!')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Run!')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Who?')
('Decoded sentence:', u'Sous de tous !\n')
-
('Input sentence:', u'Wow!')
('Decoded sentence:', u'Sous de tous !\n')
-
('Input sentence:', u'Fire!')
('Decoded sentence:', u'Atent en ait !\n')
-
('Input sentence:', u'Help!')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Jump.')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u'Atent en ait !\n')
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u'Atent en ait !\n')
-
('Input sentence:', u'Stop!')
('Decoded sentence:', u'Atent en ait !\n')
-
('Input sentence:', u'Wait!')
('Decoded sentence:', u'Atent ent !\n')
-
('Input sentence:', u'Wait!')
('Decoded sentence:', 

In [17]:
print("Time costs {} minutes.".format((time() - time_begin)/60.))

Time costs 0.471911470095 minutes.
